In [161]:
# Import needed libraries
import sys
sys.path.append(r'e:\Data science\Titanic dataset\notebooks')

from auto_imports import *


df = pd.read_csv('E:\Data science\Titanic dataset\data\Processed data\Data Analysis\cleaned data.csv')

### Feature engineering

In [162]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ind
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C23 C25 C27,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C23 C25 C27,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,C23 C25 C27,S,train


split cabin column to cabin cell and cabin number

In [163]:
df['Cabin']

0       C23 C25 C27
1               C85
2       C23 C25 C27
3              C123
4       C23 C25 C27
           ...     
1282    C23 C25 C27
1283           C105
1284    C23 C25 C27
1285    C23 C25 C27
1286    C23 C25 C27
Name: Cabin, Length: 1287, dtype: object

In [164]:
def remove_duplicate_letters(text):
        seen = set()
        result = ''
        for char in text:
            if char not in seen:
                seen.add(char)
                result += char
        return result.strip().replace(' ' , ',')

# splitting cabin column to cabin and cell number 
df['cell_number'] = df['Cabin'].replace(r'[a-zA-Z]' , '' , regex =True).str.replace(' ' , '-')
df['Cabin_letter'] = df['Cabin'].replace(r'[0-9]' , '' , regex = True).apply(remove_duplicate_letters)
# When removing digits from cabin column values output may be like that (B B B B) so I maked a function to remove this duplicted letters

In [165]:
df['Cabin_letter'].unique()

array(['C', 'E', 'G', 'D', 'A', 'B', 'F', 'F,G', 'F,E', 'T'], dtype=object)

In [166]:
df['cell_number'].unique()

array(['23-25-27', '85', '123', '46', '6', '103', '56', '78', '33', '30',
       '52', '83', '-73', '31', '5', '10-12', '26', '110', '58-60', '101',
       '-69', '47', '86', '2', '19', '7', '49', '4', '32', '80', '36',
       '15', '93', '35', '87', '77', '67', '94', '125', '99', '118', '',
       '22-26', '106', '65', '54', '57-59-63-66', '34', '18', '124', '91',
       '40', '128', '37', '50', '82', '96-98', '44', '104', '111', '92',
       '38', '21', '12', '63', '14', '20', '79', '25', '73', '95', '39',
       '22', '70', '16', '68', '10', '41', '9', '23', '48', '58', '126',
       '71', '51-53-55', '-63', '62-64', '24', '90', '45', '8', '121',
       '11', '3', '82-84', '17', '102', '69', '28', '42', '148', '61',
       '53', '43', '130', '132', '55-57', '116', '29', '51', '97', '89',
       '-46', '-57', '60', '39-41', '52-54-56', '105'], dtype=object)

In [167]:
df.drop('Cabin' , axis =1 , inplace=True)

In [168]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ind,cell_number,Cabin_letter
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,train,23-25-27,C
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,train,85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,train,23-25-27,C
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,train,123,C
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,train,23-25-27,C


I want to extract some important information from name column. Then I will drop it

In [169]:
df['Name'][0]

'Braund, Mr. Owen Harris'

- I will extract family name from the first word and title from the second word 
> Last thing I want to see names have a brackets. What do brackets refer to?

Now let's titles

In [170]:
df['Name'].str.split(r'[,.]').str[1].str.strip().value_counts()

Name
Mr              739
Miss            259
Mrs             196
Master           60
Rev               8
Dr                8
Col               4
Major             2
Ms                2
Mlle              2
Don               1
Mme               1
Lady              1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: count, dtype: int64

In [171]:
df['Title'] = df['Name'].str.split(r'[,.]').str[1].str.strip()

In [172]:
mask = df['Name'].str.contains(r'\((.*?)\)')

In [173]:
non_brackets = df.loc[~mask]
non_brackets['FamilyName'] =non_brackets['Name'].str.split(',').str[0].str.strip()

In [174]:
brackets = df.loc[mask]
brackets.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ind,cell_number,Cabin_letter,Title
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,train,85,C,Mrs
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,train,123,C,Mrs
8,9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,S,train,23-25-27,C,Mrs
9,10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,C,train,23-25-27,C,Mrs
15,16,1.0,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,S,train,23-25-27,C,Mrs


In [175]:
# Extract the text inside brackets, split by space, and get the last word for each row
brackets['FamilyName'] = brackets['Name'].str.extract(r'\((.*?)\)')[0].str.split().str[-1]

In [176]:
# combain them in comlumn called family name
df = pd.concat([non_brackets , brackets]).sort_values(by='PassengerId')
df.drop('Name' , axis = 1 , inplace=True)

In [177]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ind,cell_number,Cabin_letter,Title,FamilyName
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,S,train,23-25-27,C,Mr,Braund
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C,train,85,C,Mrs,Thayer
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,train,23-25-27,C,Miss,Heikkinen
3,4,1.0,1,female,35.0,1,0,113803,53.1000,S,train,123,C,Mrs,Peel
4,5,0.0,3,male,35.0,0,0,373450,8.0500,S,train,23-25-27,C,Mr,Allen


Now let's extract some important information from ticket column like 

1. route/agent
2. batch/timeframe
3. SharedTicket(1 , 0)

In [178]:
df['Ticket'].str.split()

0                [A/5, 21171]
1                 [PC, 17599]
2         [STON/O2., 3101282]
3                    [113803]
4                    [373450]
                ...          
1282             [A.5., 3236]
1283              [PC, 17758]
1284    [SOTON/O.Q., 3101262]
1285                 [359309]
1286                   [2668]
Name: Ticket, Length: 1287, dtype: object

In [179]:
# I want to update only_character variable
clean_agents = df['Ticket'].str.extract(r'^([A-Za-z/.]+\d*)', expand=False).fillna('Non-special').replace('[.]' , '' , regex=True)

In [180]:
clean_agents.unique()

# That's perfect

array(['A/5', 'PC', 'STON/O2', 'Non-special', 'PP', 'CA', 'SC/Paris',
       'SC/A4', 'A/4', 'SP', 'SOC', 'SO/C', 'W/C', 'SOTON/OQ', 'WEP',
       'STON/O', 'A4', 'C', 'SC/PARIS', 'SOP', 'A5', 'Fa', 'LINE', 'FCC',
       'SW/PP', 'SCO/W', 'P/PP', 'SC', 'SC/AH', 'A/S', 'WE/P', 'SO/PP',
       'FC', 'SOTON/O2', 'CA/SOTON', 'SC/A3', 'STON/OQ', 'AQ/4', 'A',
       'LP', 'AQ/3'], dtype=object)

In [181]:
df['Special Agent/route'] = clean_agents 

Make Ticket batch column

In [182]:
ticket_digits = df['Ticket'].str.split().str[-1]
ticket_digits.str.len().unique()

array([5, 7, 6, 4, 3, 1])

In [183]:
ticket_digits[ticket_digits.str.len() == 1]

758     3
825     3
1058    2
1173    2
Name: Ticket, dtype: object

In [184]:
corrected_values = df.iloc[ticket_digits[ticket_digits.str.len() == 1].index]['Ticket'].values
corrected_values

array(['S.O./P.P. 3', 'S.O./P.P. 3', 'S.O./P.P. 2', 'S.O./P.P. 2'],
      dtype=object)

In [185]:
ticket_digits[ticket_digits.str.len() == 1] = corrected_values

In [186]:
ticket_digits.str.len().unique()

array([ 5,  7,  6,  4,  3, 11])

In [187]:
ticket_digits.unique()

array(['21171', '17599', '3101282', '113803', '373450', '330877', '17463',
       '349909', '347742', '237736', '9549', '113783', '2151', '347082',
       '350406', '248706', '382652', '244373', '345763', '2649', '239865',
       '248698', '330923', '113788', '347077', '2631', '19950', '330959',
       '349216', '17601', '17569', '335677', '24579', '17604', '113789',
       '2677', '2152', '345764', '2651', '7546', '11668', '349253',
       '2123', '330958', '23567', '370371', '14311', '2662', '349237',
       '3101295', '39886', '17572', '2926', '113509', '19947', '31026',
       '2697', '34651', '2144', '2669', '36973', '347088', '17605',
       '2661', '29395', '3464', '3101281', '315151', '33111', '14879',
       '2680', '1601', '348123', '349208', '374746', '248738', '364516',
       '345767', '345779', '330932', '113059', '14885', '3101278', '6608',
       '392086', '343275', '343276', '347466', '5734', '2315', '364500',
       '374910', '17754', '17759', '231919', '244367', '349

In [188]:
df['TicketBatch'] = np.where(ticket_digits.str.len() < 5 , 1,0)

1: Early<br>
0: Late

Let's make boolean column called shared ticket

In [189]:
df['SharedTicket'] = np.where(df['Ticket'].duplicated() , 1 , 0)

In [190]:
first_digit_ticket = df['Ticket'].str.extract(r'([0-9]+)')[0]
first_digit_ticket

0             5
1         17599
2             2
3        113803
4        373450
         ...   
1282          5
1283      17758
1284    3101262
1285     359309
1286       2668
Name: 0, Length: 1287, dtype: object

In [191]:
df['Ticket'].head()

0           A/5 21171
1            PC 17599
2    STON/O2. 3101282
3              113803
4              373450
Name: Ticket, dtype: object

In [192]:
df.drop('Ticket' , axis = 1 , inplace=True)

In [193]:
df.sample(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,ind,cell_number,Cabin_letter,Title,FamilyName,Special Agent/route,TicketBatch,SharedTicket
290,297,0.0,3,male,23.500000,0,0,7.2292,C,train,23-25-27,C,Mr,Hanna,Non-special,1,0
984,1003,NaN,3,female,29.881138,0,0,7.7792,Q,test,23-25-27,C,Miss,Shine,Non-special,0,0
559,569,0.0,3,male,29.881138,0,0,7.2292,C,train,23-25-27,C,Mr,Doharr,Non-special,1,0
585,596,0.0,3,male,36.000000,1,1,24.1500,S,train,23-25-27,C,Mr,Van Impe,Non-special,0,1
40,41,0.0,3,female,40.000000,1,0,9.4750,S,train,23-25-27,C,Mrs,Larsson,Non-special,1,0
537,546,0.0,1,male,64.000000,0,0,26.0000,S,train,23-25-27,C,Mr,Nicholson,Non-special,1,0
1103,1123,NaN,1,female,21.000000,0,0,26.5500,S,test,23-25-27,C,Miss,Willard,Non-special,0,0
397,404,0.0,3,male,28.000000,1,0,15.8500,S,train,23-25-27,C,Mr,Hakkarainen,STON/O2,0,1
153,158,0.0,3,male,30.000000,0,0,8.0500,S,train,23-25-27,C,Mr,Corn,SOTON/OQ,0,0
1030,1050,NaN,1,male,42.000000,0,0,26.5500,S,test,22,D,Mr,Borebank,Non-special,0,0


In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1287 entries, 0 to 1286
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          1287 non-null   int64  
 1   Survived             875 non-null    float64
 2   Pclass               1287 non-null   int64  
 3   Sex                  1287 non-null   object 
 4   Age                  1287 non-null   float64
 5   SibSp                1287 non-null   int64  
 6   Parch                1287 non-null   int64  
 7   Fare                 1286 non-null   float64
 8   Embarked             1287 non-null   object 
 9   ind                  1287 non-null   object 
 10  cell_number          1287 non-null   object 
 11  Cabin_letter         1287 non-null   object 
 12  Title                1287 non-null   object 
 13  FamilyName           1287 non-null   object 
 14  Special Agent/route  1287 non-null   object 
 15  TicketBatch          1287 non-null   int64 

In [195]:
# saving them 
df.to_csv('E:\Data science\Titanic dataset\data\Processed data\Data Analysis\processed_data.csv' , index=False)